In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
oofs = [
    "/mnt/storage_dimm2/kaggle_output/icecube-neutrinos-in-deep-ice/20230323-102724/DynEdge/fold_0/oofs.parquet",
    "/mnt/storage_dimm2/kaggle_output/icecube-neutrinos-in-deep-ice/20230409-080525/DynEdge/fold_0/oofs.parquet",
    "/mnt/storage_dimm2/kaggle_output/icecube-neutrinos-in-deep-ice/20230405-063040/GPS/fold_0/oofs.parquet",
]

In [3]:
def angular_dist_score_numpy(az_true, zen_true, az_pred, zen_pred):
    """
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse
    cosine (arccos) thereof is then the angle between the two input vectors

    Parameters:
    -----------

    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian

    Returns:
    --------

    dist : float
        mean over the angular distance(s) in radian
    """

    if not (
        np.all(np.isfinite(az_true))
        and np.all(np.isfinite(zen_true))
        and np.all(np.isfinite(az_pred))
        and np.all(np.isfinite(zen_pred))
    ):
        raise ValueError("All arguments must be finite")

    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)

    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)

    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1 * sz2 * (ca1 * ca2 + sa1 * sa2) + (cz1 * cz2)

    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod = np.clip(scalar_prod, -1, 1)

    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))

In [4]:
azi, zen = {}, {}

for i, oof in enumerate(oofs):
    df = pd.read_parquet(oof)
    
    azi[f"model_{i}"] = df["azimuth"]
    zen[f"model_{i}"] = df["zenith"]
    

azi = pd.DataFrame(azi).values
zen = pd.DataFrame(zen).values
gt = df[["azimuth_gt", "zenith_gt"]]

In [5]:
def stack(azi, zen, weights=None):
    
    if weights is None:
        dim = azi.shape[1]
        weights = np.ones(dim) / dim
        
    azi_sin = (weights * np.sin(azi)).sum(1)
    azi_cos = (weights * np.cos(azi)).sum(1)
    
    azi_final = np.arctan2(azi_sin, azi_cos)
    zen_final = (weights * zen).sum(1)
    
    return azi_final, zen_final


# Independent weights for azi & zen
def stack_v2(azi, zen, weights=None):
    
    dim = azi.shape[1]
    
    if weights is None:
        weights = np.ones(dim) / dim
        weights = np.concatenate([weights, weights])
        
    weights_a = weights[:dim]
    weights_z = weights[dim:]
        
    azi_sin = (weights_a * np.sin(azi)).sum(1)
    azi_cos = (weights_a * np.cos(azi)).sum(1)
    
    azi_final = np.arctan2(azi_sin, azi_cos)
    zen_final = (weights_z * zen).sum(1)
    
    return azi_final, zen_final

In [6]:
def scorer(weights=None, v2=False):
    if v2:
        azi_final, zen_final = stack_v2(azi, zen, weights)
    else:
        azi_final, zen_final = stack(azi, zen, weights)
    return angular_dist_score_numpy(gt["azimuth_gt"], gt["zenith_gt"], azi_final, zen_final)

In [7]:
scorer()

0.9832772158461447

In [8]:
scorer(v2=True)

0.9832772158461447

In [9]:
dim = len(oofs)
weights = np.ones(dim) / dim

In [10]:
minimize(scorer, weights)

      fun: 0.9830387844521334
 hess_inv: array([[ 0.0057585 ,  0.01026726,  0.00094249],
       [ 0.01026726,  0.03273609, -0.00214129],
       [ 0.00094249, -0.00214129,  0.00116897]])
      jac: array([-0.0014438 ,  0.00131305,  0.00503559])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 364
      nit: 27
     njev: 88
   status: 2
  success: False
        x: array([0.44303154, 0.29891726, 0.25539217])

In [11]:
minimize(scorer, np.concatenate([weights, weights]), args=(True,))

      fun: 0.983015440620221
 hess_inv: array([[  3.82926418,  -0.14212013,  -1.00748414,  12.00131169,
         -6.05945958,  -5.96633318],
       [ -0.14212013,   3.07382121,   0.18793389,   8.63632281,
         -2.7867009 ,  -5.88012781],
       [ -1.00748414,   0.18793389,   1.55727577,  -1.85556015,
         -2.13665998,   4.01635516],
       [ 12.00131169,   8.63632281,  -1.85556015, 109.76540464,
        -77.97046415, -31.8965659 ],
       [ -6.05945958,  -2.7867009 ,  -2.13665998, -77.97046415,
        103.50286258, -25.58937244],
       [ -5.96633318,  -5.88012781,   4.01635516, -31.8965659 ,
        -25.58937244,  57.7024627 ]])
      jac: array([ 5.21540642e-07,  2.05636024e-06, -2.67475843e-06,  1.26659870e-06,
        1.33365393e-06,  1.75088644e-06])
  message: 'Optimization terminated successfully.'
     nfev: 287
      nit: 33
     njev: 41
   status: 0
  success: True
        x: array([0.44820087, 0.25852994, 0.28477468, 0.50602103, 0.25648293,
       0.23482001])